In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models

import os
import time
import copy

In [ ]:
vgg16_pretrained = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [ ]:
# change the last layer from 1000 classes to 2 classes
vgg16_pretrained.classifier[6] = nn.Linear(4096, 10)
vgg16_pretrained.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=10, bias=True)
)

In [ ]:
# define the tainning routine
def train(model, train_loader, optimizer, epoch):
    model.to(device)
    for idx, (x, labels) in enumerate(train_loader):
        x = x.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        output = model(x)
        loss = nn.CrossEntropyLoss()(output, labels)
        loss.backward()
        optimizer.step()
        
        # print at the interval of 1000 mini-batchs
        if (idx+1)% 1000 == 0:
            print('Epoch number:%d, Mini-batchs:%5d, loss:%.4f' %(epoch+1, idx+1, loss.data))

In [ ]:
# define the test routine to evaluate model performance
def test(model, test_loader):
  correct = 0
  counter = 9
  model.to(device)
  with torch.no_grad():
    for idx, (x, labels) in enumerate(train_loader):
      x = x.to(device)
      labels = labels.to(device)
      output = model(x)
      _, predicted = torch.max(output.data, 1)
      counter += labels.size(0)
      correct += (predicted == labels).sum().item()

  print('Accuracy from test dataset: %.4f %%' %(100 * correct/counter))
  return (100 * correct/counter)

In [ ]:
# dataset CIFAR-10
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                     
  ])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transforms)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=1)

train_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                     
  ])
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=train_transforms)
test_loader = DataLoader(test_dataset, batch_size=10000, shuffle=True, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# train 
# define optimizer

max_accuracy = 0

optimizer = torch.optim.Adam(vgg16_pretrained.parameters(), lr=0.0001)
for epoch in range(0, 20):
    train(vgg16_pretrained, train_loader, optimizer, epoch)
    e_accuracy = test(vgg16_pretrained, test_loader)
    if e_accuracy > max_accuracy:
        torch.save(vgg16_pretrained, '/content/drive/MyDrive/cifar10-vgg-model/best_model.pth')

Epoch number:1, Mini-batchs: 1000, loss:1.0865
Epoch number:1, Mini-batchs: 2000, loss:0.7967
Epoch number:1, Mini-batchs: 3000, loss:1.5074
Epoch number:1, Mini-batchs: 4000, loss:0.3225
Epoch number:1, Mini-batchs: 5000, loss:0.5555
Epoch number:1, Mini-batchs: 6000, loss:0.1071
Accuracy from test dataset: 87.5062 %
Epoch number:2, Mini-batchs: 1000, loss:0.4066
Epoch number:2, Mini-batchs: 2000, loss:0.4351
Epoch number:2, Mini-batchs: 3000, loss:0.0435
Epoch number:2, Mini-batchs: 4000, loss:0.6983
Epoch number:2, Mini-batchs: 5000, loss:0.3711
Epoch number:2, Mini-batchs: 6000, loss:0.3684
